In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import re
%matplotlib inline

In [8]:
# Konfigurasi
video_path = 'D:\\TA_Lalu_Lintas\\TA-Lalu-Lintas\\dataset-mentah\\vidios\\TC_00007(1).mov'  # Path ke video

# Direktori untuk dataset YOLO
output_yolo_images = 'dataset_yolo/images/'
output_yolo_labels = 'dataset_yolo/labels/'
os.makedirs(output_yolo_images, exist_ok=True)
os.makedirs(output_yolo_labels, exist_ok=True)

# Direktori untuk dataset CNN (gambar kepala)
output_cnn_worn = 'dataset_cnn/helmet_worn/'
output_cnn_not_worn = 'dataset_cnn/helmet_not_worn/'
os.makedirs(output_cnn_worn, exist_ok=True)
os.makedirs(output_cnn_not_worn, exist_ok=True)

In [4]:
# Variabel Global
drawing = False
ix, iy = -1, -1
boxes = []  # Menyimpan semua bounding box per frame
current_class = 0  # 0 = Motor, 1 = Helmet
frame_count = 0
paused = False  # Status video (berjalan atau berhenti)
# FRAMES PER SECOND FOR VIDEO
fps = 30

In [5]:
# Fungsi menggambar bounding box
def draw_bbox(event, x, y, flags, param):
    global ix, iy, drawing, img, boxes, current_class

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()
            color = (0, 255, 0) if current_class == 0 else (255, 0, 0)
            cv2.rectangle(img_copy, (ix, iy), (x, y), color, 2)
            cv2.imshow('Frame', img_copy)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        color = (0, 255, 0) if current_class == 0 else (255, 0, 0)
        cv2.rectangle(img, (ix, iy), (x, y), color, 2)
        x_min, y_min = min(ix, x), min(iy, y)
        x_max, y_max = max(ix, x), max(iy, y)
        boxes.append((current_class, x_min, y_min, x_max, y_max))
        cv2.imshow('Frame', img)

In [6]:
# Fungsi untuk mendapatkan nomor urut terakhir dari file dalam sebuah folder
def get_last_file_number(folder_path, prefix, extension):
    """
    Mengambil nomor urut terakhir dari file dengan prefix tertentu dalam folder.
    Contoh file: 'frame_0001.txt' untuk label YOLO atau 'head_0001.jpg' untuk gambar kepala
    
    Args:
    - folder_path: Lokasi folder tempat file disimpan
    - prefix: Prefix dari file (misalnya 'frame' atau 'head')
    - extension: Ekstensi file (misalnya '.txt' untuk label atau '.jpg' untuk gambar)
    
    Returns:
    - nomor urut terakhir yang ditemukan, jika tidak ada file maka 0
    """
    # Ambil semua file dalam folder
    files = os.listdir(folder_path)
    
    # Filter file yang memiliki prefix dan ekstensi tertentu
    pattern = re.compile(f'{prefix}_(\d+){extension}')  # Filter dengan ekstensi sesuai (txt untuk label, jpg untuk gambar)
    numbers = []
    
    for file in files:
        match = pattern.match(file)
        if match:
            numbers.append(int(match.group(1)))  # Ambil nomor urut dari nama file
    
    # Jika tidak ada file yang sesuai, kembalikan 0
    if not numbers:
        return 0
    
    # Kembalikan nomor urut terbesar
    return max(numbers)


In [9]:
# Load video
cap = cv2.VideoCapture(video_path)
cv2.namedWindow('Frame')
cv2.setMouseCallback('Frame', draw_bbox)

frame_count = 0
paused = False
boxes = []  # Temporary list for storing current bounding boxes
saved_bboxes = set()  # Set to store already saved bounding boxes

while cap.isOpened():
    if not paused:
        time.sleep(1/fps)
        ret, frame = cap.read()
        if not ret:
            print("Video selesai!")
            break
        img = frame.copy()
        cv2.imshow('Frame', img)

    key = cv2.waitKey(1)

    if key == 32:  # Space untuk pause/play
        paused = not paused

    elif key == ord('m'):  # Motor
        current_class = 0

    elif key == ord('h'):  # Helmet
        current_class = 1

    elif key == ord('r'):  # Reset label
        img = frame.copy()
        boxes.clear()

    elif key == ord('n'):  # Next frame tanpa menyimpan
        paused = False
        boxes.clear()

    elif key == ord('q'):  # Keluar program
        cap.release()
        cv2.destroyAllWindows()
        exit()

    elif key == 13:  # Simpan data jika ada label
        if len(boxes) == 0:
            print("Tidak ada label, frame tidak disimpan!")
        else:
            # Dapatkan nomor terakhir dari folder
            last_label_number = get_last_file_number(output_yolo_labels, "frame", ".txt")  # Untuk label YOLO
            last_image_number = get_last_file_number(output_yolo_images, "frame", ".jpg")  # Untuk gambar YOLO
            last_worn = get_last_file_number(output_cnn_worn, "head", ".jpg")  # Untuk helm yang dikenakan
            last_not_worn = get_last_file_number(output_cnn_not_worn, "head", ".jpg")  # Untuk helm yang tidak dikenakan

            # Simpan label dalam format YOLO
            label_file = os.path.join(output_yolo_labels, f'frame_{last_label_number + 1:04d}.txt')  # Perbaikan di sini
            with open(label_file, 'w') as f:
                motor_bboxes = []  # Menyimpan semua bounding box motor
                helmet_bboxes = []  # Menyimpan semua bounding box helm

                # Pisahkan motor dan helm
                for class_id, x_min, y_min, x_max, y_max in boxes:
                    # Memeriksa apakah bounding box sudah pernah disimpan
                    bbox_key = (class_id, x_min, y_min, x_max, y_max)
                    if bbox_key in saved_bboxes:
                        continue  # Skip jika sudah disimpan

                    # Konversi koordinat ke format YOLO
                    x_center = (x_min + x_max) / 2 / frame.shape[1]
                    y_center = (y_min + y_max) / 2 / frame.shape[0]
                    width = (x_max - x_min) / frame.shape[1]
                    height = (y_max - y_min) / frame.shape[0]
                    f.write(f'{class_id} {x_center} {y_center} {width} {height}\n')

                    if class_id == 0:  # Motor
                        motor_bboxes.append((x_min, y_min, x_max, y_max))
                    elif class_id == 1:  # Helmet
                        helmet_bboxes.append((x_min, y_min, x_max, y_max))

                    # Tandai bounding box sebagai sudah disimpan
                    saved_bboxes.add(bbox_key)

            # Simpan gambar YOLO
            image_file = os.path.join(output_yolo_images, f'frame_{last_image_number + 1:04d}.jpg')
            cv2.imwrite(image_file, frame)

            # Ekstrak kepala untuk dataset CNN
            for motor_bbox in motor_bboxes:
                x_min, y_min, x_max, y_max = motor_bbox
                head_crop = frame[y_min:y_min + int(0.3 * (y_max - y_min)), x_min:x_max]

                # Memeriksa apakah helm ada di dalam bounding box motor
                helmet_found = False
                for helmet_bbox in helmet_bboxes:
                    hx_min, hy_min, hx_max, hy_max = helmet_bbox
                    if hx_min >= x_min and hy_min >= y_min and hx_max <= x_max and hy_max <= y_max:
                        helmet_found = True
                        break

                # Simpan gambar CNN berdasarkan status helm
                if helmet_found:
                    last_worn += 1
                    filename = f'head_{last_worn:04d}.jpg'
                    cv2.imwrite(os.path.join(output_cnn_worn, filename), head_crop)
                else:
                    last_not_worn += 1
                    filename = f'head_{last_not_worn:04d}.jpg'
                    cv2.imwrite(os.path.join(output_cnn_not_worn, filename), head_crop)

            frame_count += 1
            paused = False

cap.release()
cv2.destroyAllWindows()


Video selesai!
